In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [ ]:
from torch_geometric.datasets import KarateClub

In [ ]:
import torch_geometric.transforms as T
dataset= KarateClub(transform=T.NormalizeFeatures())
data = dataset[0]
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-5)
criterion = torch.nn.NLLLoss()



torch.manual_seed(42)
perm = torch.randperm(data.num_nodes)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[perm[:20]] = True 
data.y = data.y 
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

model.eval()
pred = out.argmax(dim=1)
correct = (pred == data.y).sum()
acc = int(correct) / data.num_nodes
print(f'Acurácia total: {acc:.2%}')